Updated to remove outliers using a hybrid Z-score (features) + IQR (target) rule.

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, mean_squared_error

train_path = "MiNDAT.csv"
test_path = "MiNDAT_UNK.csv"
target_col = "CORRUCYSTIC_DENSITY"
id_col = "LOCAL_IDENTIFIER"
feature_cols = ["jNhEum"]  # exact column names

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

X = train_df[feature_cols].apply(pd.to_numeric, errors="coerce")
y = pd.to_numeric(train_df[target_col], errors="coerce")
X_test = test_df[feature_cols].apply(pd.to_numeric, errors="coerce")

mask_y = y.notna()
X, y = X.loc[mask_y].copy(), y.loc[mask_y].copy()

train_medians = X.median(numeric_only=True).fillna(0.0)
X = X.fillna(train_medians)

row_mask = X.notna().all(axis=1)
X, y = X.loc[row_mask], y.loc[row_mask]

# Remove outliers using a hybrid Z-score (features) + IQR (target) rule
# - Z-score threshold on features: 3.0
# - IQR factor on target: 1.5 (Tukey's rule)
# This typically trims extreme rows that destabilize RMSE.
feat_mean = X.mean()
feat_std = X.std(ddof=0).replace(0, 1.0)  # guard against zero std to avoid inf
z_scores = (X - feat_mean) / feat_std
z_keep = (z_scores.abs() <= 3.0).all(axis=1)

q1, q3 = y.quantile([0.25, 0.75])
iqr = q3 - q1
lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
y_keep = y.between(lower, upper)

outlier_keep_mask = z_keep & y_keep
removed = len(X) - int(outlier_keep_mask.sum())
if removed > 0:
    print(
        f"[Outlier filter] Removed {removed} rows "
        f"({removed / len(outlier_keep_mask) * 100:.2f}% of cleaned training)."
    )
X, y = X.loc[outlier_keep_mask].copy(), y.loc[outlier_keep_mask].copy()

# Replace NaNs with the same training medians
X_test = X_test.fillna(train_medians)
X_test = X_test.fillna(0.0)

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

pipe = make_pipeline(
    StandardScaler(with_mean=True, with_std=True),
    PolynomialFeatures(degree=2, include_bias=False),
    Ridge(alpha=1.0),  # removed random_state; Ridge doesn't use it
)


def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


pipe.fit(X, y)
poly = pipe.named_steps["polynomialfeatures"]
ridge_est = pipe.named_steps["ridge"]

from sklearn.model_selection import GridSearchCV


def get_poly_feature_names(poly, input_features):
    return poly.get_feature_names_out(input_features)


# If X is a DataFrame, pass its columns; otherwise build generic names
input_features = (
    list(X.columns)
    if hasattr(X, "columns")
    else [f"x{i}" for i in range(np.shape(X)[1])]
)
feature_names = poly.get_feature_names_out()

# 5) Map coefficients to names
coefs = pd.Series(ridge_est.coef_, index=feature_names, name="coefficient")

print("\nTop-weighted polynomial features (after standardization):")
print(coefs.reindex(coefs.abs().sort_values(ascending=False).index).head(20))
print("\nIntercept:", ridge_est.intercept_)

rmse_scorer = make_scorer(rmse, greater_is_better=False)

# Cross-validation setup
rkf = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)

# Hyperparameter grid: tune polynomial degree and ridge alpha
param_grid = {
    "polynomialfeatures__degree": [1, 2, 3],
    "ridge__alpha": np.logspace(-3, 3, 13),
}

grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring=rmse_scorer,  # negative RMSE internally
    cv=rkf,
    n_jobs=None,
    refit=True,  # refit the best pipeline on full data
)

# Run the search (optimizing RMSE)
grid.fit(X, y)

best_pipe = grid.best_estimator_
best_rmse = -grid.best_score_
print(f"Best params: {grid.best_params_}")
print(f"CV RMSE (best): mean={best_rmse:.4f}")

# Also print CV R² using the best pipeline
r2_scores = cross_val_score(best_pipe, X, y, scoring="r2", cv=rkf, n_jobs=None)
print(f"CV R² (best): mean={np.mean(r2_scores):.4f}, std={np.std(r2_scores):.4f}")

# Fit on full cleaned training data
best_pipe.fit(X, y)

# Inspect learned coefficients
ridge_est = best_pipe.named_steps["ridge"]
poly = best_pipe.named_steps["polynomialfeatures"]  # match step name
feature_names = poly.get_feature_names_out()
coefs = pd.Series(ridge_est.coef_, index=feature_names, name="coefficient")
print("\nTop-weighted polynomial features (after standardization):")
print(coefs.sort_values(key=np.abs, ascending=False).head(20))

# Predict and build submission
preds = best_pipe.predict(X_test)
submission = pd.DataFrame({id_col: test_df[id_col].values, target_col: preds})[
    [id_col, target_col]
]

out_file = "corrucystic_density_predictions16.csv"
submission.to_csv(out_file, index=False)
print(f"\nWrote submission file: {out_file}")